In [1]:
%matplotlib qt

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from poisson_fem import PoissonFEM
import ROM
import GenerativeSurrogate as gs
import Data as dta
import numpy as np
import scipy.sparse as sps
import scipy.sparse.linalg as lg
import time
import petsc4py
import sys
petsc4py.init(sys.argv)
from petsc4py import PETSc
import torch
from torch import optim

/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/constantin/anaconda3/envs/genDRROM/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/c

In [2]:
# Some parameters
lin_dim_rom = 4                      # Linear number of rom elements
a = np.array([1, 1, 0])              # Boundary condition function coefficients
dtype = torch.float                  # Tensor data type
supervised_samples = {n for n in range(16)}
unsupervised_samples = {n for n in range(16, 1024)}

In [3]:
# Define mesh and boundary conditions
mesh = PoissonFEM.RectangularMesh(np.ones(lin_dim_rom)/lin_dim_rom)
# mesh.plot()

def origin(x):
    return np.abs(x[0]) < np.finfo(float).eps and np.abs(x[1]) < np.finfo(float).eps

def ess_boundary_fun(x):
    return 0.0
mesh.set_essential_boundary(origin, ess_boundary_fun)

def domain_boundary(x):
    # unit square
    return np.abs(x[0]) < np.finfo(float).eps or np.abs(x[1]) < np.finfo(float).eps or \
            np.abs(x[0]) > 1.0 - np.finfo(float).eps or np.abs(x[1]) > 1.0 - np.finfo(float).eps
mesh.set_natural_boundary(domain_boundary)

def flux(x):
    q = np.array([a[0] + a[2]*x[1], a[1] + a[2]*x[0]])
    return q

In [4]:
#Specify right hand side and stiffness matrix
rhs = PoissonFEM.RightHandSide(mesh)
rhs.set_natural_rhs(mesh, flux)
K = PoissonFEM.StiffnessMatrix(mesh)
rhs.set_rhs_stencil(mesh, K)

In [5]:
trainingData = dta.StokesData(supervised_samples, unsupervised_samples)
trainingData.read_data()
# trainingData.plotMicrostruct(1)
trainingData.reshape_microstructure_image()

In [6]:
# define rom
rom = ROM.ROM(mesh, K, rhs, trainingData.output_resolution**2)

In [7]:
model = gs.GenerativeSurrogate(rom, trainingData)
# model = gs.GenerativeSurrogate()
# model.load()

In [ ]:
model.fit(n_steps=10, with_precisions=False)

step =  0
loss z =  46552460.0
loss z =  46551940.0
loss z =  46551512.0
loss z =  46551092.0
loss z =  46550676.0
loss z =  46550300.0
loss z =  46549948.0
loss z =  46549588.0
loss z =  46549280.0
z step =  66.22704935073853 s
loss_f =  22717092.0
loss_f =  22338676.0
loss_f =  22042946.0
loss_f =  21786820.0
loss_f =  21562504.0
loss_f =  21358004.0
loss_f =  21166976.0
loss_f =  20993110.0
loss_f =  20850312.0
thetaf step =  42.831541299819946 s
sample ==  0
loss_lambda_c =  93169718919168.0
sample ==  1
loss_lambda_c =  53483138973696.0
sample ==  2
loss_lambda_c =  813718732537856.0
sample ==  3
loss_lambda_c =  952551201570816.0
sample ==  4
loss_lambda_c =  859595023056896.0
sample ==  5
loss_lambda_c =  1598079752994816.0
sample ==  6
loss_lambda_c =  483804074475520.0
sample ==  7
loss_lambda_c =  248794067238912.0
sample ==  8
loss_lambda_c =  64371019481088.0
sample ==  9
loss_lambda_c =  213646235926528.0
sample ==  10
loss_lambda_c =  391692091916288.0
sample ==  11
loss_

In [ ]:
model.fit(n_steps=50, with_precisions=True)

In [ ]:
model.save()

In [ ]:
model = gs.GenerativeSurrogate()
model.load()

In [ ]:
test_samples = {n for n in range(0, 8)}
testData = dta.StokesData(unsupervised_samples=test_samples)
testData.read_data()
# trainingData.plotMicrostruct(1)
testData.reshape_microstructure_image()

In [ ]:
uf_pred, Z = model.predict(testData)

In [ ]:
uf_pred[4]

In [ ]:
x = torch.tensor((float('nan')))

In [ ]:
torch.isnan(x)

In [ ]:
x = math.nan

In [ ]:
not torch.isnan(x)

In [ ]:
model.fit(n_steps=10)

In [ ]:
with torch.no_grad():
    log_lambdac_pred = model.pcNet(model.z_mean[:model.data.n_supervised_samples])
    model.tauc = torch.tensor(model.batch_size_N_thetac, dtype=model.dtype) / \
                torch.sum((log_lambdac_pred.squeeze() - model.log_lambdac_mean) ** 2, dim=0)

In [ ]:
model.tauc

In [ ]:
with torch.no_grad():
    eps = 1e-10
    uf_pred = torch.zeros_like(model.data.P)
    for n in range(model.data.n_supervised_samples):
        uf_pred[n] = model.rom_autograd(torch.exp(model.log_lambdac_mean[n]))
        model.taucf = torch.tensor(model.data.n_supervised_samples, dtype=model.dtype) / \
                                                    torch.sum((uf_pred - model.data.P) ** 2 + eps, dim=0)

In [ ]:
model.taucf

In [ ]:
model.rom.full_solution.array

In [ ]:
x = np.outer(np.linspace(0, 1, 129), np.ones(129))
y = x.copy().T # transpose

In [ ]:
fig = plt.figure()

In [ ]:
ax = plt.axes(projection='3d')

In [ ]:
ax.plot_surface(x, y, np.reshape(model.rom.interpolated_solution.array, (129, 129)),cmap='viridis', edgecolor='none')

In [ ]:
pp = np.reshape(model.data.P[15, :], (129, 129))

In [ ]:
ax.plot_surface(x, y, pp.detach().numpy())

In [ ]:
model.rom.full_solution.array

In [ ]:
model.taucf

In [ ]:
model.log_lambda_c_mean

In [ ]:
True % 10

In [ ]:
rom.solution.array

In [ ]:
y = torch.tensor([1, 2, 3])

In [ ]:
y  + 1.2

In [ ]:
model.tauc